# Fine-Tuning a BERT Model and Create a Text Classifier

We have already performed the Feature Engineering to create BERT embeddings from the `reviews_body` text using the pre-trained BERT model, and split the dataset into train, validation and test files. To optimize for Tensorflow training, we saved the files in TFRecord format. 

Now, let’s fine-tune the BERT model to our Customer Reviews Dataset and add a new classification layer to predict the `star_rating` for a given `review_body`.

![BERT Training](img/bert_training.png)

As mentioned earlier, BERT’s attention mechanism is called a Transformer. This is, not coincidentally, the name of the popular BERT Python library, “Transformers,” maintained by a company called [HuggingFace](https://github.com/huggingface/transformers). We will use a variant of BERT called [DistilBert](https://arxiv.org/pdf/1910.01108.pdf) which requires less memory and compute, but maintains very good accuracy on our dataset.

# DEMO 3: 

# Run Model Training on Amazon SageMaker

Amazon SageMaker is a fully managed service that provides every developer and data scientist with the ability to build, train, and deploy machine learning (ML) models quickly. SageMaker removes the heavy lifting from each step of the machine learning process to make it easier to develop high quality models.

In [1]:
!pip install -q sagemaker==2.16.3

In [2]:
import boto3
import sagemaker

session   = sagemaker.Session()
bucket = session.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

# Specify the Dataset in S3
We are using the train, validation, and test splits created in the previous section.

In [3]:
processed_train_data_s3_uri='s3://fsx-container-demo/input/data/train'

!aws s3 ls $processed_train_data_s3_uri/

2020-11-22 17:21:24      10728 part-algo-1-amazon_reviews_us_Digital_Software_v1_00.tfrecord
2020-11-22 17:21:24      11812 part-algo-2-amazon_reviews_us_Digital_Video_Games_v1_00.tfrecord


In [4]:
processed_validation_data_s3_uri='s3://fsx-container-demo/input/data/validation'

!aws s3 ls $processed_validation_data_s3_uri/

2020-11-22 17:21:24        679 part-algo-1-amazon_reviews_us_Digital_Software_v1_00.tfrecord
2020-11-22 17:21:24        642 part-algo-2-amazon_reviews_us_Digital_Video_Games_v1_00.tfrecord


In [5]:
processed_test_data_s3_uri='s3://fsx-container-demo/input/data/test'

!aws s3 ls $processed_test_data_s3_uri/

2020-11-22 17:21:24        615 part-algo-1-amazon_reviews_us_Digital_Software_v1_00.tfrecord
2020-11-22 17:21:24        632 part-algo-2-amazon_reviews_us_Digital_Video_Games_v1_00.tfrecord


# Specify S3 `Distribution Strategy`

In [6]:
from sagemaker.inputs import TrainingInput

s3_input_train_data = TrainingInput(s3_data=processed_train_data_s3_uri, 
                                         distribution='ShardedByS3Key') 
s3_input_validation_data = TrainingInput(s3_data=processed_validation_data_s3_uri, 
                                              distribution='ShardedByS3Key')
s3_input_test_data = TrainingInput(s3_data=processed_test_data_s3_uri, 
                                        distribution='ShardedByS3Key')

print(s3_input_train_data.config)
print(s3_input_validation_data.config)
print(s3_input_test_data.config)

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://fsx-container-demo/input/data/train', 'S3DataDistributionType': 'ShardedByS3Key'}}}
{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://fsx-container-demo/input/data/validation', 'S3DataDistributionType': 'ShardedByS3Key'}}}
{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://fsx-container-demo/input/data/test', 'S3DataDistributionType': 'ShardedByS3Key'}}}


# Setup Hyper-Parameters for Classification Layer

In [7]:
epochs=3
learning_rate=0.00001
epsilon=0.00000001
train_batch_size=128
validation_batch_size=64
test_batch_size=64
train_steps_per_epoch=100
validation_steps=10
test_steps=10
use_xla=True
use_amp=True
max_seq_length=64
freeze_bert_layer=True
run_validation=True
run_test=True
run_sample_predictions=True

# Setup Metrics To Track Model Performance

These sample log lines...
```
45/50 [=====>..] - ETA: 3s - loss: 0.425 - accuracy: 0.881
50/50 [=======>] - ETA: 0s - val_loss: 0.407 - val_accuracy: 0.885
```
...will produce the following 4 metrics in CloudWatch:

`loss` = 0.425

`accuracy` = 0.881

`val_loss` = 0.407

`val_accuracy` = 0.885

<img src="img/cloudwatch_train_accuracy.png" width="50%" align="left">

<img src="img/cloudwatch_train_loss.png" width="50%" align="left">

In [8]:
metrics_definitions = [
     {'Name': 'train:loss', 'Regex': 'loss: ([0-9\\.]+)'},
     {'Name': 'train:accuracy', 'Regex': 'accuracy: ([0-9\\.]+)'},
     {'Name': 'validation:loss', 'Regex': 'val_loss: ([0-9\\.]+)'},
     {'Name': 'validation:accuracy', 'Regex': 'val_accuracy: ([0-9\\.]+)'},
]

# Setup Our BERT + TensorFlow Script to Run on SageMaker
Prepare our TensorFlow model to run on the managed SageMaker service

In [9]:
!pygmentize ./code/train.py

import time
import random
import pandas as pd
from glob import glob
import pprint
import argparse
import json
import subprocess
import sys
import os
import tensorflow as tf

subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'transformers==2.8.0'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'sagemaker-tensorflow==2.1.0.1.0.0'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'scikit-learn==0.23.1'])

from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
from transformers import TextClassificationPipeline
from transformers.configuration_distilbert import DistilBertConfig
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

CLASSES = [1, 2, 3, 4, 5]


def select_data_and_label_from_record(record):
    x = {
        'input_ids': record['input_ids'],
        'input_mask': record['input_mask'],
        'segment_ids': record['segment_ids']
   

In [10]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(entry_point='train.py',
                       source_dir='code',
                       role=role,
                       instance_count=1,
                       instance_type='ml.c5.9xlarge',
                       use_spot_instances=True,
                       max_run=3600,
                       max_wait=3600,
                       volume_size=1024,
                       py_version='py3',
                       framework_version='2.1.0',
                       hyperparameters={'epochs': epochs,
                                        'learning_rate': learning_rate,
                                        'epsilon': epsilon,
                                        'train_batch_size': train_batch_size,
                                        'validation_batch_size': validation_batch_size,
                                        'test_batch_size': test_batch_size,                                             
                                        'train_steps_per_epoch': train_steps_per_epoch,
                                        'validation_steps': validation_steps,
                                        'test_steps': test_steps,
                                        'use_xla': use_xla,
                                        'use_amp': use_amp,                                             
                                        'max_seq_length': max_seq_length,
                                        'freeze_bert_layer': freeze_bert_layer,                                     
                                        'run_validation': run_validation,
                                        'run_test': run_test,
                                        'run_sample_predictions': run_sample_predictions},
                       input_mode='Pipe',
                       metric_definitions=metrics_definitions
                      )

# Train the Model on SageMaker

In [11]:
estimator.fit(inputs={'train': s3_input_train_data, 
                      'validation': s3_input_validation_data,
                      'test': s3_input_test_data
              },                                 
              wait=False)

In [12]:
training_job_name = estimator.latest_training_job.name
print('Training Job Name:  {}'.format(training_job_name))

Training Job Name:  tensorflow-training-2020-11-23-12-48-53-685


In [13]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/jobs/{}">Training Job</a> After About 5 Minutes</b>'.format(region, training_job_name)))


In [14]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/TrainingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(region, training_job_name)))


In [15]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Training Job Has Completed</b>'.format(bucket, training_job_name, region)))


In [16]:
%%time

estimator.latest_training_job.wait(logs=False)


2020-11-23 12:48:54 Starting - Starting the training job
2020-11-23 12:48:55 Starting - Launching requested ML instances.............
2020-11-23 12:50:05 Starting - Preparing the instances for training......
2020-11-23 12:50:44 Downloading - Downloading input data...
2020-11-23 12:51:00 Training - Downloading the training image..
2020-11-23 12:51:17 Training - Training image download completed. Training in progress.................................................................................................................................................................................................................................................
2020-11-23 13:11:28 Uploading - Uploading generated training model...........
2020-11-23 13:12:31 Completed - Training job completed
CPU times: user 1.04 s, sys: 88.3 ms, total: 1.13 s
Wall time: 23min 37s


# Wait Until the ^^ Training Job ^^ Completes Above!

In [17]:
!aws s3 ls s3://$bucket/$training_job_name/output/

2020-11-23 13:12:23  498712704 model.tar.gz
